# Data Preparation

In [81]:
import pandas as pd # load and manipulate data
import numpy as np # calculation
import matplotlib.pyplot as plt # visualize data
from sklearn.preprocessing import OneHotEncoder # To one-hot encode variables
import category_encoders as ce # To target encode high cardinality variables

## Load raw dataset

In [82]:
demand = pd.read_csv('shopee_raw_data.csv')

## Overview

In [83]:
print(demand.head().to_string())

       date   product_id                                                                product_name           product_category    brand  traffic  impressions  payment  revenue  product_ad_spend  shop_ad_spend  auto_ad_spend  run_shop_ad  run_product_ad  product_page_bounce_count  traffic_from_search  wm_yr_wk  month  d  doubleday  near_dday  end_of_month  weekend  other_commercial_sale  day_offs  avg_price  promotion_on  promotion_price price_bin discount_rate  comment_received  product_rating  high_rating  high_discount  high_comment  avg_category_rate  avg_category_comment  week_of_month  wday
0  5/1/2024  10070408774  Bình Ða D?ng 0.75L - Nh?p kh?u chính hãng 100% t? thuong hi?u Melitta, Ð?c  6. Other Melitta products  Melitta        0            0        0        0                 0             67          30000            1               0                          0                    0    202417      5  1          0          0             0        0                      0       

In [84]:
def check_unique_values(df):
    unique_values = {col: df[col].nunique() for col in df.columns}
    result_df = pd.DataFrame(list(unique_values.items()), columns=["Column Name", "Unique Value Count"])
    return result_df

check_unique_values(demand)

,Column Name,Unique Value Count
0,date,214
1,product_id,131
2,product_name,133
3,product_category,20
4,brand,13
5,traffic,56
6,impressions,97
7,payment,12
8,revenue,1185
9,product_ad_spend,567


In [85]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28676 entries, 0 to 28675
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       28676 non-null  object 
 1   product_id                 28676 non-null  int64  
 2   product_name               28676 non-null  object 
 3   product_category           28676 non-null  object 
 4   brand                      28676 non-null  object 
 5   traffic                    28676 non-null  int64  
 6   impressions                28676 non-null  int64  
 7   payment                    28676 non-null  int64  
 8   revenue                    28676 non-null  int64  
 9   product_ad_spend           28676 non-null  int64  
 10  shop_ad_spend              28676 non-null  int64  
 11  auto_ad_spend              28676 non-null  int64  
 12  run_shop_ad                28676 non-null  int64  
 13  run_product_ad             28676 non-null  int

Seems like we have some issues:
* Misalignment between `product_id` and `product_name`
* Some variables in the wrong data types

## Standardize data types

`date` should be in datetime data type

In [86]:
# Ensure the 'date' column is in datetime format
demand['date'] = pd.to_datetime(demand['date'])

# Sort the data by date, then product_id
demand = demand.sort_values(by=['product_id', 'date'])

`discount_rate` should be float   

`product_id` should be string

In [87]:
# Convert 'discount_rate' to float
demand['discount_rate'] = pd.to_numeric(
    demand['discount_rate'].str.replace('%', '', regex=False), 
    errors='coerce'
).astype(float).round(4)

# Convert 'product_id' to string
demand['product_id'] = demand['product_id'].astype(str)

Double-check

In [88]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28676 entries, 25252 to 25251
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       28676 non-null  datetime64[ns]
 1   product_id                 28676 non-null  object        
 2   product_name               28676 non-null  object        
 3   product_category           28676 non-null  object        
 4   brand                      28676 non-null  object        
 5   traffic                    28676 non-null  int64         
 6   impressions                28676 non-null  int64         
 7   payment                    28676 non-null  int64         
 8   revenue                    28676 non-null  int64         
 9   product_ad_spend           28676 non-null  int64         
 10  shop_ad_spend              28676 non-null  int64         
 11  auto_ad_spend              28676 non-null  int64         
 12  run_s

In [89]:
print(demand.head().to_string())

            date  product_id                                                                                    product_name     product_category    brand  traffic  impressions  payment  revenue  product_ad_spend  shop_ad_spend  auto_ad_spend  run_shop_ad  run_product_ad  product_page_bounce_count  traffic_from_search  wm_yr_wk  month  d  doubleday  near_dday  end_of_month  weekend  other_commercial_sale  day_offs  avg_price  promotion_on  promotion_price price_bin  discount_rate  comment_received  product_rating  high_rating  high_discount  high_comment  avg_category_rate  avg_category_comment  week_of_month  wday
25252 2024-05-01  3388329772  Combo 12 h?p cà phê viên nén Carraro  - Nh?p kh?u t? Ý - Tuong thích v?i máy capsule Nespresso  3. Carraro capsules  Carraro        0            0        0        0                 0             67          30000            1               1                          0                    0    202417      5  1          0          0             0  

## Fix misalignment between `product_id` and `product_name`

In [90]:
def check_row_misalignment(df, col1, col2):
    misaligned = df.groupby(col1)[col2].nunique().reset_index()
    misaligned = misaligned[misaligned[col2] > 1]
    return misaligned

misaligned_rows = check_row_misalignment(demand, 'product_id', 'product_name')
print(misaligned_rows)

misaligned_product_ids = misaligned_rows['product_id'].tolist()

print("Check these product_ids for misalignment between product_id and product_name:")
print(misaligned_product_ids)

     product_id  product_name
43  15765129329             2
76  21382736254             2
81  22150796991             2
Check these product_ids for misalignment between product_id and product_name:
['15765129329', '21382736254', '22150796991']


In [91]:
filtered_by_product_id = demand[demand['product_id'].isin(misaligned_product_ids)]

# Check the shape of the DataFrame
rows, columns = filtered_by_product_id.shape

print(f"Number of rows: {rows}")
print(f"Number of columns: {columns}")

# Export data to check in excel
filtered_by_product_id.to_csv('filtered_by_product_id.csv')

Number of rows: 1284
Number of columns: 39


I examined the data and see that there are dates where `product_name` A in `product_id` 1 had data. Within that same date, `product_name` B corresponding to the same `product_id` has 0 for its variables. And vice versa. So I'll keep rows where the `product_id` has "meaningful data" (non-zero values for its important variables), and eliminate rows where the `product_id` carries 0 for its variables (impressions, traffic, etc.) in another `product_name` for the same `date`.

This is because in November, the sales admin changed the content of the product pages: keeping high performing `product_id` (meaning the `product_id` contains high rating, high comment volumes, etc), merging low performing products into that `product_id` and then changed the name of the corresponding `product_name`.

In [92]:
# Filter the DataFrame for misaligned product_ids
filtered_demand = demand[demand['product_id'].isin(misaligned_product_ids)]

# Case 1: Remove duplicate rows based on the specified subset of columns
filtered_demand = filtered_demand.drop_duplicates(subset=['date', 'product_id', 'traffic', 'impressions', 'payment'])

# Case 2: After Case 1, check for each combination of date & product_id, if there are 2 rows,
# keep the row with impressions > 0 or payment > 0, else remove the row with impressions = 0.
def filter_impressions_and_payment(group):
    if len(group) == 2:  # Ensure there are exactly 2 rows per combination
        # Check for rows where impressions = 0 and the other row has either impressions > 0 or payment > 0
        if (group['impressions'] == 0).any() and ((group['impressions'] > 0).any() or (group['payment'] > 0).any()):
            # Keep the row where impressions > 0 or payment > 0
            group = group[(group['impressions'] > 0) | (group['payment'] > 0)]
    return group

# Apply the filtering logic only to the filtered DataFrame
filtered_demand = filtered_demand.groupby(['date', 'product_id'], group_keys=False).apply(filter_impressions_and_payment)

/tmp/ipykernel_4840/3250136266.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_demand = filtered_demand.groupby(['date', 'product_id'], group_keys=False).apply(filter_impressions_and_payment)


In [93]:
# Group by 'product_id' and count the number of rows for each
date_counts = filtered_demand.groupby('product_id')['date'].count()

# Set display option to show all rows
pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

# Print the total counts for each product_id
print("Total rows (all dates) for each product_id:")
print(date_counts)

Total rows (all dates) for each product_id:
product_id
15765129329    214
21382736254    214
22150796991    214
Name: date, dtype: int64


It's aligned now.

In [94]:
# Update the original DataFrame with the modified rows for misaligned product_ids
# First, drop the rows for the misaligned product_ids from the original DataFrame
demand = demand[~demand['product_id'].isin(misaligned_product_ids)]

# Then, append the filtered DataFrame back
demand = pd.concat([demand, filtered_demand], ignore_index=True)